<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/BERT_on_IEMOCAP_for_DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import re
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam  
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import pickle
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.utils.np_utils import to_categorical  
import time

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2
from keras.applications.resnet50 import ResNet50

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
import pickle

In [2]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece

In [3]:
import tokenization

In [4]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [5]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = pooled_output
    clf_output1 = tf.reshape(clf_output,[32,32,3])
    base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape=clf_output1.shape)
    print("Hi")
    # x = base_model.output
    # x = GlobalAveragePooling2D()(x)
    out = Dense(6, activation='softmax')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model,clf_output1

In [6]:
# base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (32,32,3))
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# predictions = Dense(6 , activation= 'softmax')(x)
# model = Model(inputs = base_model.input, outputs = predictions)

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 39.2 s, sys: 12 s, total: 51.2 s
Wall time: 3min 14s


In [8]:
with open('/content/gdrive/My Drive/iemocap/train/sentences.pkl', 'rb') as f:
    data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/train/labels.pkl', 'rb') as f:
    labels = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/sentences.pkl', 'rb') as f:
    test_data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/labels.pkl', 'rb') as f:
    test_labels = pickle.load(f)

In [9]:
def preprocessing(data,labels):
  processed_data=[]
  processed_label=[]
  for i in range(0,len(data)):
    for j in range(0,len(data[i])):
      intermediate_data=[]
      intermediate_label=[]
      for k in range(0,len(data[i][j])):
        text=data[i][j][k]
        if text != '<eos>'and text!='<pad>':
          intermediate_data.append(text)
      processed_data.append(intermediate_data)
  for i in labels:
    for j in i:
      processed_label.append(j)
  return processed_data,processed_label

In [10]:
processed_data,processed_label = preprocessing(data,labels)
test_processed_data,test_processed_label = preprocessing(test_data,test_labels)

In [11]:
for i in range(0,len(processed_data)):
  processed_data[i]= ' '.join(processed_data[i])
for i in range(0,len(test_processed_data)):
  test_processed_data[i]=' '.join(test_processed_data[i])

In [12]:
processed_data_batching=processed_data

In [13]:
processed_data=np.asarray(processed_data)
test_processed_data=np.asarray(test_processed_data)
Y=to_categorical(processed_label, num_classes=6)
test_Y=to_categorical(test_processed_label, num_classes=6)

In [14]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [15]:
t0 = time.time()
train_input = bert_encode(processed_data, tokenizer, max_len=160)
test_input = bert_encode(test_processed_data, tokenizer, max_len=160)
train_labels = Y
test_labels = test_Y

In [16]:
model,clf_output1 = build_model(bert_layer, max_len=161)
model.summary()

Hi
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 161)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 161)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 161)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]   

In [ ]:
train_history = model.fit(
    train_input, train_labels,
    epochs=1,
    batch_size=1
)

model.save('model.h5')

3088/4699 [==================>...........] - ETA: 4:37 - loss: 1.5513 - accuracy: 0.3782

**SAVING THE INTERMEDIATE LAYER OUTPUT**

In [18]:
count=0
batch=[]
batch_processed_data=[]
for i in processed_data:
  if count!=50:
    batch.append(i)
    count=count+1
  else:
    batch_processed_data.append(batch)
    batch=[]
    count=0
batch_processed_data.append(batch)

In [64]:
print(len(batch_processed_data))
print((processed_data))

93
['yeah and they said get whatever you want and ...', 'mmmm some miso soup .', 'some miso soup . you can get whatever . you can get a vat full of miso soup . god .', 'ah man .', 'this is going to be so great . i have so many great ideas .', "it took them , oh man , it took them like , i mean what it 's like april now and any letters that come in april are usually", 'oh my god . i am so proud of you .']


In [19]:
train_labels = Y
layer_names=[layer.name for layer in model.layers]
layer_name=layer_names[3]
intermediate_encodings=[]
for processed_data in batch_processed_data:
  train_input = bert_encode(processed_data, tokenizer, max_len=160)
  intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
  intermediate_output = intermediate_layer_model.predict(train_input)
  intermediate_encodings.append(intermediate_output)
print(len(intermediate_encodings))

93


In [20]:
model = ResNet50(include_top=False, input_shape=(32,32,3), weights='imagenet')

94773248/94765736 [==============================] - 1s 0us/step


In [35]:
intermediate_encodings_in_np = np.array(intermediate_encodings)
inter_reshaped=intermediate_encodings_in_np[0][0][0].reshape(32,32)
inter_expanded=np.repeat(inter_reshaped[..., np.newaxis],3,-1)

In [61]:
print(len(intermediate_encodings))

93


In [23]:
y_pred=model.predict(test_input, batch_size=1)

In [24]:
y_pred1=np.argmax(y_pred,axis=1)
test_processed_label=np.asarray(test_processed_label)
t1 = time.time()
total = t1-t0
print("Total Execution time (Training + Testing): " + str(total))
print("Accuracy : " + str(accuracy_score(test_processed_label, y_pred1)))
print("Weighted F1-score : " + str(f1_score(test_processed_label, y_pred1, average='weighted')))

Total Execution time (Training + Testing): 948.8990435600281
Accuracy : 0.5169439309919901
Weighted F1-score : 0.5111446336164024
